In [5]:
import numpy as np
import PIL
import os
import glob
import json
from geomet import wkt
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import matplotlib.pyplot as plt
from tqdm import tqdm


In [6]:
inputfol = ["./xview2_geotiff/geotiffs/hold/labels/","./xview2_geotiff/geotiffs/test/labels/","./xview2_geotiff/geotiffs/tier1/labels/","./xview2_geotiff/geotiffs/tier3/labels/"]
outputpostfol = ["./Dataset/Validation/Post/Label512/","./Dataset/Test/Post/Label512/","./Dataset/Tier1/Post/Label512/","./Dataset/Tier3/Post/Label512/"]
outputprefol = ["./Dataset/Validation/Pre/Label512/","./Dataset/Test/Pre/Label512/","./Dataset/Tier1/Pre/Label512/","./Dataset/Tier3/Pre/Label512/"]

In [7]:
def createlabels(inputfol,outputpostfol,outputprefol,size):
    for i in range(len(inputfol)):
        inputpath = inputfol[i]
        outputpost = outputpostfol[i]
        outputpre = outputprefol[i]
        json_list = []
        for filename in os.listdir(inputpath):
            if filename.endswith('.json'):
                data = open(inputpath + filename)
                json_list.append((json.load(data),filename))
        with tqdm(total=len(json_list), desc='Saving to ' + outputpost + ' and ' + outputpre, unit='img') as pbar:
            for js in json_list:
                width = js[0]['metadata']['width']
                height = js[0]['metadata']['height']
                image = Image.new("L", (width, height), color = (0))
                draw = ImageDraw.Draw(image)
                if('pre_disaster' in js[1]):
                    for j in js[0]['features']['xy']:
                        polygonlist = wkt.loads(j['wkt'])
                        polygontuples = [(x,y) for x,y in polygonlist['coordinates'][0]]
                        draw.polygon((polygontuples), fill=(1))
                    image = image.resize(size,PIL.Image.NEAREST)
                    image.save(outputpre+js[1].split('.')[0] + '.png')
                if('post_disaster' in js[1]):
                    for j in js[0]['features']['xy']:
                        subtype = j['properties']['subtype']
                        fill = (0)
                        polygonlist = wkt.loads(j['wkt'])
                        polygontuples = [(x,y) for x,y in polygonlist['coordinates'][0]]
                        match subtype:
                            case "no-damage":
                                fill = (1)
                            case "minor-damage":
                                fill = (2)
                            case "major-damage":
                                fill = (3)
                            case "destroyed":
                                fill = (4)
                            case _:
                                fill = (0)
                        draw.polygon((polygontuples), fill=fill)
                    image = image.resize(size,PIL.Image.NEAREST)
                    image.save(outputpost+js[1].split('.')[0] + '.png')
                pbar.update(1)

In [8]:
createlabels(inputfol,outputpostfol,outputprefol,(512,512))

Saving to ./Dataset/Validation/Post/Label512/ and ./Dataset/Validation/Pre/Label512/: 100%|█| 1866/1866 [00:24<00:00, 7
Saving to ./Dataset/Test/Post/Label512/ and ./Dataset/Test/Pre/Label512/: 100%|███| 1866/1866 [00:23<00:00, 79.33img/s]
Saving to ./Dataset/Tier1/Post/Label512/ and ./Dataset/Tier1/Pre/Label512/: 100%|█| 5598/5598 [01:11<00:00, 78.55img/s]
Saving to ./Dataset/Tier3/Post/Label512/ and ./Dataset/Tier3/Pre/Label512/: 100%|█| 12738/12738 [01:23<00:00, 152.31img
